#001-setup
### DO NOT MODIFY - RUN "AS IS"

In [0]:
%pip install pandas
dbutils.library.restartPython()

In [0]:
import pandas as pd

In [0]:
catalog = "users"
schema = "david_hurley"

In [0]:
spark.sql(f"""CREATE CATALOG IF NOT EXISTS {catalog}""")
spark.sql(f"""CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}""")

In [0]:
# create Delta table for non-partitioned csv
csv_to_create_tables_for = [
    "alberta_wdrill_boreholes.csv",
    "alberta_wdrill_driller_drilling_company.csv",
    "alberta_wdrill_drillers.csv",
    "alberta_wdrill_drilling_companies.csv",
    "alberta_wdrill_other_seals.csv",
    "alberta_wdrill_perforations.csv",
    "alberta_wdrill_pump_test_items.csv",
    "alberta_wdrill_screens.csv",
    "alberta_wdrill_wells.csv",
    "alberta_wtest_bacterio_analysis_info.csv",
    "alberta_wtest_bacterio_details.csv",
    "alberta_wtest_exceedances.csv",
    "alberta_wtest_field_notes.csv",
    "alberta_wtest_field_personnel.csv",
    "alberta_wtest_gas_analysis_info.csv",
    "alberta_wtest_gas_details.csv",
    "alberta_wtest_gcdwq_standard.csv",
    "alberta_wtest_iso_analysis_info.csv",
    "alberta_wtest_iso_details.csv",
    "alberta_wtest_well.csv",
    "alberta_wtest_well_measures.csv",
    "alberta_wtest_well_test.csv",
    "alberta_wtest_wqual_analysis_info.csv",
    "alberta_wtest_wqual_details.csv",
    "alberta_wtest_yield_hour",
    "alberta_wtest_yield_test.csv",
    "alberta_wtest_yield_test_field_parameters.csv"
]

for filename in csv_to_create_tables_for:
    df = pd.read_csv(f"../data/{filename}", low_memory=False)
    spark_df = spark.createDataFrame(df)
    spark_df.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.{filename.split(".csv")[0]}")
    print(f"Created {catalog}.{schema}.{filename}")

In [0]:
# create Delta table for partitioned lithology
df1 = pd.read_csv("../data/alberta_wdrill_lithologies_1.csv", low_memory=False)
df2 = pd.read_csv("../data/alberta_wdrill_lithologies_2.csv", low_memory=False)
combined_df = pd.concat([df1, df2], ignore_index=True, sort=False)

spark_df = spark.createDataFrame(combined_df)
spark_df.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.alberta_wdrill_lithologies")
print(f"Created {catalog}.{schema}.alberta_wdrill_lithologies")

In [0]:
# create Delta table for partitioned well reports
df1 = pd.read_csv("../data/alberta_wdrill_well_reports_1.csv", low_memory=False)
df2 = pd.read_csv("../data/alberta_wdrill_well_reports_2.csv", low_memory=False)
df3 = pd.read_csv("../data/alberta_wdrill_well_reports_3.csv", low_memory=False)
df4 = pd.read_csv("../data/alberta_wdrill_well_reports_4.csv", low_memory=False)
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True, sort=False)

spark_df = spark.createDataFrame(combined_df)
spark_df.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.alberta_wdrill_well_reports")
print(f"Created {catalog}.{schema}.alberta_wdrill_well_reports")

In [0]:
# create Delta table for well location and material depth combined - for easy spatial sql analysis. Group material to common names

sql = f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.alberta_wdrill_combined_location_lithology AS
SELECT 
    row_number() OVER (ORDER BY t1.Well_ID, t3.Depth) AS id,  -- auto-increment ID
    t1.Well_ID,
    t1.Longitude, 
    t1.Latitude, 
    t3.Depth AS Depth_Of_Material,
    t3.Water_Bearing, 
    t3.Material,
    CASE
        WHEN t3.Material ILIKE '%Clay%' THEN 'Clay'
        WHEN t3.Material ILIKE '%Sand%' THEN 'Sand'
        WHEN t3.Material ILIKE '%Gravel%' THEN 'Gravel'
        WHEN t3.Material ILIKE '%Silt%' THEN 'Silt'
        WHEN t3.Material ILIKE '%Shale%' THEN 'Shale'
        WHEN t3.Material ILIKE '%Coal%' THEN 'Coal'
        WHEN t3.Material ILIKE '%Rock%' 
          OR t3.Material ILIKE '%Bedrock%' 
          OR t3.Material ILIKE '%Limestone%' 
          OR t3.Material ILIKE '%Granite%' 
          OR t3.Material ILIKE '%Sandstone%' THEN 'Rock'
        WHEN t3.Material ILIKE '%Till%' THEN 'Till'
        WHEN t3.Material ILIKE '%Topsoil%' THEN 'Topsoil'
        WHEN t3.Material ILIKE '%Organic%' 
          OR t3.Material ILIKE '%Muskeg%' 
          OR t3.Material ILIKE '%Tarsand%' THEN 'Organic'
        ELSE 'Other'
    END AS Material_Category
FROM {catalog}.{schema}.alberta_wdrill_wells AS t1
INNER JOIN {catalog}.{schema}.alberta_wdrill_well_reports AS t2
    ON t1.Well_ID = t2.Well_ID
INNER JOIN {catalog}.{schema}.alberta_wdrill_lithologies AS t3
    ON t2.Well_Report_ID = t3.Well_Report_ID
"""
spark.sql(sql)


In [0]:
# create Delta table for test well location and water quality combined - for easy spatial sql analysis

sql = f"""
    CREATE OR REPLACE TABLE {catalog}.{schema}.alberta_wtest_well_combined_location_quality AS (
    SELECT 
        t1.WELL_ID,
        t1.LONGITUDE,
        t1.LATTITUDE,
        t4.PARAMETER_NAME,
        t4.PARAMETER_VALUE
    FROM {catalog}.{schema}.alberta_wtest_well AS t1
    INNER JOIN {catalog}.{schema}.alberta_wtest_well_test AS t2 
    ON t1.WELL_ID = t2.WELL_ID
    INNER JOIN {catalog}.{schema}.alberta_wtest_wqual_details AS t3
    ON t2.WELL_TEST_ID = t3.WELL_TEST_ID
    INNER JOIN {catalog}.{schema}.alberta_wtest_wqual_analysis_info AS t4
    ON t3.WQUAL_DETAILS_ID = t4.WQUAL_DETAILS_ID
    )
"""
spark.sql(sql)